## Elastic Net Model

Libraries and Datasets:

In [15]:
from sklearn.linear_model import ElasticNet,ElasticNetCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from scipy import stats
import numpy as np
import pandas as pd

In [3]:
xtest = pd.read_csv('../data/processed/X_test_norm.csv')
x_train = pd.read_csv('../data/processed/X_train_norm.csv')
y_train = pd.read_csv('../data/processed/y_train_norm.csv')

x_test = xtest.drop(columns = 'Id')

### Initial Model Implementation

Looking for reasonable start point for alpha

In [12]:
alphas = [1e-5, 0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 0.7, 1, 10]

for a in alphas:
    elastic_mdl = ElasticNet(alpha=a).fit(x_train,y_train)   
    score = elastic_mdl.score(x_train,y_train)
    pred_y = model.predict(x_train)
    mse = mean_squared_error(y_train, pred_y)   
    print("Alpha:{0:.5f}, R2:{1:.3f}, MSE:{2:.3f}, RMSE:{3:.3f}"
       .format(a, score, mse, np.sqrt(mse)))

Alpha:0.00001, R2:0.878, MSE:0.035, RMSE:0.187
Alpha:0.00010, R2:0.878, MSE:0.035, RMSE:0.187
Alpha:0.00100, R2:0.872, MSE:0.035, RMSE:0.187
Alpha:0.01000, R2:0.843, MSE:0.035, RMSE:0.187
Alpha:0.10000, R2:0.790, MSE:0.035, RMSE:0.187
Alpha:0.30000, R2:0.787, MSE:0.035, RMSE:0.187
Alpha:0.50000, R2:0.786, MSE:0.035, RMSE:0.187
Alpha:0.70000, R2:0.784, MSE:0.035, RMSE:0.187
Alpha:1.00000, R2:0.781, MSE:0.035, RMSE:0.187
Alpha:10.00000, R2:0.669, MSE:0.035, RMSE:0.187


In [10]:
elastic=ElasticNet(alpha=0.001).fit(x_train, y_train)


In [11]:
elastic_cv=ElasticNetCV(alphas=alphas, cv=5)
elastic_mdl = elastic_cv.fit(x_train, y_train)
print(elastic_mdl.alpha_)
print(elastic_mdl.intercept_)

0.0001
-11.436009730604985


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:1100: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.706659657939877, tolerance: 0.01826408902830186
  tol, rng, random, positive)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.5971586659663313, tolerance: 0.01826408902830186
  tol, rng, random, positive)


In [47]:
elastic_cv.mse_path_

array([[0.14933776, 0.1795121 , 0.16017762, 0.14663854, 0.14643197],
       [0.14933776, 0.1795121 , 0.16017762, 0.14663854, 0.14643197],
       [0.11477127, 0.14422776, 0.12520469, 0.11025767, 0.1112188 ],
       [0.06168486, 0.07918688, 0.06979833, 0.05844972, 0.0587114 ],
       [0.02432067, 0.02877119, 0.02814334, 0.0248464 , 0.02372475],
       [0.01171897, 0.0141506 , 0.01494081, 0.01280785, 0.01239556],
       [0.0127709 , 0.01537937, 0.01538305, 0.01232849, 0.01390394],
       [0.01424486, 0.01668973, 0.01698854, 0.01295832, 0.01507925],
       [0.01458307, 0.01698473, 0.01744188, 0.01310881, 0.01524352]])

In [49]:
elastic_cv.score(x_train,y_train)

0.9263529313032638

## Hyperparameter tuning

Looking to find best combination of alpha and l1_ratio

In [28]:
elasticnet_mdl = ElasticNet(warm_start = True, max_iter = 1e7)

Define ranges of parameters for initial broad search

In [29]:
alphas = alphas = [1e-5, 0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 0.7, 1, 10]
l1_ratios = np.linspace(0.1,1,10)
elastic_params = {'alpha': alphas,'l1_ratio':l1_ratios}

In [30]:
#GridSearch
elastic_param_search = GridSearchCV(estimator = elasticnet_mdl,
                                    param_grid = elastic_params,
                                    cv = 10,
                                    return_train_score = True,
                                    verbose =2)

In [31]:
elastic_param_search.fit(x_train,y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV] alpha=1e-05, l1_ratio=0.1 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.1, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.1 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.1, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.1 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.1, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.1 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.1, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.1 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.1, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.1 .......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ........................ alpha=1e-05, l1_ratio=0.1, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.1 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.1, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.1 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.1, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.1 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.1, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.1 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.1, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.2 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.2, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.2 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.2, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.2 .......................................
[CV] .

[CV] ......... alpha=1e-05, l1_ratio=0.7000000000000001, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.7000000000000001 ........................
[CV] ......... alpha=1e-05, l1_ratio=0.7000000000000001, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.7000000000000001 ........................
[CV] ......... alpha=1e-05, l1_ratio=0.7000000000000001, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.7000000000000001 ........................
[CV] ......... alpha=1e-05, l1_ratio=0.7000000000000001, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.7000000000000001 ........................
[CV] ......... alpha=1e-05, l1_ratio=0.7000000000000001, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.7000000000000001 ........................
[CV] ......... alpha=1e-05, l1_ratio=0.7000000000000001, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.8 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.8, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.8 .......................................
[CV] .

[CV] ....... alpha=0.0001, l1_ratio=0.30000000000000004, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.30000000000000004 ......................
[CV] ....... alpha=0.0001, l1_ratio=0.30000000000000004, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.30000000000000004 ......................
[CV] ....... alpha=0.0001, l1_ratio=0.30000000000000004, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.30000000000000004 ......................
[CV] ....... alpha=0.0001, l1_ratio=0.30000000000000004, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.30000000000000004 ......................
[CV] ....... alpha=0.0001, l1_ratio=0.30000000000000004, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.30000000000000004 ......................
[CV] ....... alpha=0.0001, l1_ratio=0.30000000000000004, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.30000000000000004 ......................
[CV] ....... alpha=0.0001, l1_ratio=0.30000000000000004, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.30000000000000004 ......................
[CV] .

...

[CV] ............................ alpha=1, l1_ratio=0.5, total=   0.0s
[CV] alpha=1, l1_ratio=0.5 ...........................................
[CV] ............................ alpha=1, l1_ratio=0.5, total=   0.0s
[CV] alpha=1, l1_ratio=0.5 ...........................................
[CV] ............................ alpha=1, l1_ratio=0.5, total=   0.0s
[CV] alpha=1, l1_ratio=0.5 ...........................................
[CV] ............................ alpha=1, l1_ratio=0.5, total=   0.0s
[CV] alpha=1, l1_ratio=0.6 ...........................................
[CV] ............................ alpha=1, l1_ratio=0.6, total=   0.0s
[CV] alpha=1, l1_ratio=0.6 ...........................................
[CV] ............................ alpha=1, l1_ratio=0.6, total=   0.0s
[CV] alpha=1, l1_ratio=0.6 ...........................................
[CV] ............................ alpha=1, l1_ratio=0.6, total=   0.0s
[CV] alpha=1, l1_ratio=0.6 ...........................................
[CV] .

[CV] ........................... alpha=10, l1_ratio=0.1, total=   0.0s
[CV] alpha=10, l1_ratio=0.1 ..........................................
[CV] ........................... alpha=10, l1_ratio=0.1, total=   0.0s
[CV] alpha=10, l1_ratio=0.1 ..........................................
[CV] ........................... alpha=10, l1_ratio=0.1, total=   0.0s
[CV] alpha=10, l1_ratio=0.1 ..........................................
[CV] ........................... alpha=10, l1_ratio=0.1, total=   0.0s
[CV] alpha=10, l1_ratio=0.1 ..........................................
[CV] ........................... alpha=10, l1_ratio=0.1, total=   0.0s
[CV] alpha=10, l1_ratio=0.2 ..........................................
[CV] ........................... alpha=10, l1_ratio=0.2, total=   0.0s
[CV] alpha=10, l1_ratio=0.2 ..........................................
[CV] ........................... alpha=10, l1_ratio=0.2, total=   0.0s
[CV] alpha=10, l1_ratio=0.2 ..........................................
[CV] .

[CV] ............ alpha=10, l1_ratio=0.7000000000000001, total=   0.0s
[CV] alpha=10, l1_ratio=0.7000000000000001 ...........................
[CV] ............ alpha=10, l1_ratio=0.7000000000000001, total=   0.0s
[CV] alpha=10, l1_ratio=0.8 ..........................................
[CV] ........................... alpha=10, l1_ratio=0.8, total=   0.0s
[CV] alpha=10, l1_ratio=0.8 ..........................................
[CV] ........................... alpha=10, l1_ratio=0.8, total=   0.0s
[CV] alpha=10, l1_ratio=0.8 ..........................................
[CV] ........................... alpha=10, l1_ratio=0.8, total=   0.0s
[CV] alpha=10, l1_ratio=0.8 ..........................................
[CV] ........................... alpha=10, l1_ratio=0.8, total=   0.0s
[CV] alpha=10, l1_ratio=0.8 ..........................................
[CV] ........................... alpha=10, l1_ratio=0.8, total=   0.0s
[CV] alpha=10, l1_ratio=0.8 ..........................................
[CV] .

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   35.2s finished


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
                                  l1_ratio=0.5, max_iter=10000000.0,
                                  normalize=False, positive=False,
                                  precompute=False, random_state=None,
                                  selection='cyclic', tol=0.0001,
                                  warm_start=True),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [1e-05, 0.0001, 0.001, 0.01, 0.1, 0.3, 0.5,
                                   0.7, 1, 10],
                         'l1_ratio': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring=None, verbose=2)

In [32]:
elastic_param_search.best_params_

{'alpha': 0.0001, 'l1_ratio': 1.0}

In [33]:
elastic_param_search.cv_results_['mean_test_score'].max()

0.849226439985751

### Refine search

In [66]:
alphas2 = np.linspace(0.00005,0.001,100)
l1_ratios2 = np.linspace(0.01,1,20)
elastic_params2 = {'alpha': alphas2,'l1_ratio':l1_ratios2}

In [67]:
elastic_param_search2 = GridSearchCV(estimator = elasticnet_mdl,
                                    param_grid = elastic_params2,
                                    cv = 10,
                                    return_train_score = True,
                                    verbose =2)

In [68]:
elastic_param_search2.fit(x_train,y_train)

Fitting 10 folds for each of 2000 candidates, totalling 20000 fits
[CV] alpha=5e-05, l1_ratio=0.01 ......................................
[CV] ....................... alpha=5e-05, l1_ratio=0.01, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.01 ......................................
[CV] ....................... alpha=5e-05, l1_ratio=0.01, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.01 ......................................
[CV] ....................... alpha=5e-05, l1_ratio=0.01, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.01 ......................................
[CV] ....................... alpha=5e-05, l1_ratio=0.01, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.01 ......................................
[CV] ....................... alpha=5e-05, l1_ratio=0.01, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.01 ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ....................... alpha=5e-05, l1_ratio=0.01, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.01 ......................................
[CV] ....................... alpha=5e-05, l1_ratio=0.01, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.01 ......................................
[CV] ....................... alpha=5e-05, l1_ratio=0.01, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.01 ......................................
[CV] ....................... alpha=5e-05, l1_ratio=0.01, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.01 ......................................
[CV] ....................... alpha=5e-05, l1_ratio=0.01, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.06210526315789474 .......................
[CV] ........ alpha=5e-05, l1_ratio=0.06210526315789474, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.06210526315789474 .......................
[CV] ........ alpha=5e-05, l1_ratio=0.06210526315789474, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.06210526315789474 .......................
[CV] .

[CV] ........ alpha=5e-05, l1_ratio=0.32263157894736844, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.32263157894736844 .......................
[CV] ........ alpha=5e-05, l1_ratio=0.32263157894736844, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.37473684210526315 .......................
[CV] ........ alpha=5e-05, l1_ratio=0.37473684210526315, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.37473684210526315 .......................
[CV] ........ alpha=5e-05, l1_ratio=0.37473684210526315, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.37473684210526315 .......................
[CV] ........ alpha=5e-05, l1_ratio=0.37473684210526315, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.37473684210526315 .......................
[CV] ........ alpha=5e-05, l1_ratio=0.37473684210526315, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.37473684210526315 .......................
[CV] ........ alpha=5e-05, l1_ratio=0.37473684210526315, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.37473684210526315 .......................
[CV] .

[CV] ......... alpha=5e-05, l1_ratio=0.6352631578947369, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.6352631578947369 ........................
[CV] ......... alpha=5e-05, l1_ratio=0.6352631578947369, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.6873684210526316 ........................
[CV] ......... alpha=5e-05, l1_ratio=0.6873684210526316, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.6873684210526316 ........................
[CV] ......... alpha=5e-05, l1_ratio=0.6873684210526316, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.6873684210526316 ........................
[CV] ......... alpha=5e-05, l1_ratio=0.6873684210526316, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.6873684210526316 ........................
[CV] ......... alpha=5e-05, l1_ratio=0.6873684210526316, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.6873684210526316 ........................
[CV] ......... alpha=5e-05, l1_ratio=0.6873684210526316, total=   0.0s
[CV] alpha=5e-05, l1_ratio=0.6873684210526316 ........................
[CV] .

...

[CV] ........ alpha=0.001, l1_ratio=0.16631578947368422, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.16631578947368422 .......................
[CV] ........ alpha=0.001, l1_ratio=0.16631578947368422, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.16631578947368422 .......................
[CV] ........ alpha=0.001, l1_ratio=0.16631578947368422, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.16631578947368422 .......................
[CV] ........ alpha=0.001, l1_ratio=0.16631578947368422, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.16631578947368422 .......................
[CV] ........ alpha=0.001, l1_ratio=0.16631578947368422, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.16631578947368422 .......................
[CV] ........ alpha=0.001, l1_ratio=0.16631578947368422, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.16631578947368422 .......................
[CV] ........ alpha=0.001, l1_ratio=0.16631578947368422, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.16631578947368422 .......................
[CV] .

[CV] ......... alpha=0.001, l1_ratio=0.4789473684210527, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.4789473684210527 ........................
[CV] ......... alpha=0.001, l1_ratio=0.4789473684210527, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.4789473684210527 ........................
[CV] ......... alpha=0.001, l1_ratio=0.4789473684210527, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.5310526315789474 ........................
[CV] ......... alpha=0.001, l1_ratio=0.5310526315789474, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.5310526315789474 ........................
[CV] ......... alpha=0.001, l1_ratio=0.5310526315789474, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.5310526315789474 ........................
[CV] ......... alpha=0.001, l1_ratio=0.5310526315789474, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.5310526315789474 ........................
[CV] ......... alpha=0.001, l1_ratio=0.5310526315789474, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.5310526315789474 ........................
[CV] .

[CV] ......... alpha=0.001, l1_ratio=0.8436842105263158, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.8436842105263158 ........................
[CV] ......... alpha=0.001, l1_ratio=0.8436842105263158, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.8436842105263158 ........................
[CV] ......... alpha=0.001, l1_ratio=0.8436842105263158, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.8436842105263158 ........................
[CV] ......... alpha=0.001, l1_ratio=0.8436842105263158, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.8436842105263158 ........................
[CV] ......... alpha=0.001, l1_ratio=0.8436842105263158, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.8436842105263158 ........................
[CV] ......... alpha=0.001, l1_ratio=0.8436842105263158, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.8436842105263158 ........................
[CV] ......... alpha=0.001, l1_ratio=0.8436842105263158, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.8436842105263158 ........................
[CV] .

[Parallel(n_jobs=1)]: Done 20000 out of 20000 | elapsed:  9.8min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
                                  l1_ratio=0.5, max_iter=10000000.0,
                                  normalize=False, positive=False,
                                  precompute=False, random_state=None,
                                  selection='cyclic', tol=0.0001,
                                  warm_start=True),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([5.00000000e-05, 5.95959596e-05, 6.91919192e...
       9.71212121e-04, 9.80808081e-04, 9.90404040e-04, 1.00000000e-03]),
                         'l1_ratio': array([0.01      , 0.06210526, 0.11421053, 0.16631579, 0.21842105,
       0.27052632, 0.32263158, 0.37473684, 0.42684211, 0.47894737,
       0.53105263, 0.58315789, 0.63526316, 0.68736842, 0.73947368,
       0.79157895, 0.84368421, 0.89578947, 0.94789474, 1.        ])},
             pre_dispatch='2*n_jobs',

In [42]:
elastic_param_search2.best_params_

{'alpha': 0.00020510204081632653, 'l1_ratio': 0.5310526315789474}

In [43]:
elastic_param_search2.cv_results_['mean_test_score'].max()

0.849280997475811

Second tuning

In [51]:
alphas3 = np.linspace(0.0001,0.0003,50)
l1_ratios3 = np.linspace(0.001, 1,20)
elastic_params3 = {'alpha': alphas,'l1_ratio':l1_ratios}

In [56]:
elastic_param_search3 = GridSearchCV(estimator = elasticnet_mdl,
                                    param_grid = elastic_params3,
                                    cv = 10,
                                    return_train_score = True,
                                    verbose =2)

In [57]:
elastic_param_search3.fit(x_train,y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV] alpha=1e-05, l1_ratio=0.1 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.1, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.1 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.1, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.1 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.1, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.1 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.1, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.1 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.1, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.1 .......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ........................ alpha=1e-05, l1_ratio=0.1, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.1 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.1, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.1 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.1, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.1 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.1, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.1 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.1, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.2 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.2, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.2 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.2, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.2 .......................................
[CV] .

[CV] ......... alpha=1e-05, l1_ratio=0.7000000000000001, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.7000000000000001 ........................
[CV] ......... alpha=1e-05, l1_ratio=0.7000000000000001, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.7000000000000001 ........................
[CV] ......... alpha=1e-05, l1_ratio=0.7000000000000001, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.7000000000000001 ........................
[CV] ......... alpha=1e-05, l1_ratio=0.7000000000000001, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.8 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.8, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.8 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.8, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.8 .......................................
[CV] ........................ alpha=1e-05, l1_ratio=0.8, total=   0.0s
[CV] alpha=1e-05, l1_ratio=0.8 .......................................
[CV] .

[CV] ....... alpha=0.0001, l1_ratio=0.30000000000000004, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.4 ......................................
[CV] ....................... alpha=0.0001, l1_ratio=0.4, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.4 ......................................
[CV] ....................... alpha=0.0001, l1_ratio=0.4, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.4 ......................................
[CV] ....................... alpha=0.0001, l1_ratio=0.4, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.4 ......................................
[CV] ....................... alpha=0.0001, l1_ratio=0.4, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.4 ......................................
[CV] ....................... alpha=0.0001, l1_ratio=0.4, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.4 ......................................
[CV] ....................... alpha=0.0001, l1_ratio=0.4, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.4 ......................................
[CV] .

...

[CV] ............................ alpha=1, l1_ratio=0.6, total=   0.0s
[CV] alpha=1, l1_ratio=0.6 ...........................................
[CV] ............................ alpha=1, l1_ratio=0.6, total=   0.0s
[CV] alpha=1, l1_ratio=0.7000000000000001 ............................
[CV] ............. alpha=1, l1_ratio=0.7000000000000001, total=   0.0s
[CV] alpha=1, l1_ratio=0.7000000000000001 ............................
[CV] ............. alpha=1, l1_ratio=0.7000000000000001, total=   0.0s
[CV] alpha=1, l1_ratio=0.7000000000000001 ............................
[CV] ............. alpha=1, l1_ratio=0.7000000000000001, total=   0.0s
[CV] alpha=1, l1_ratio=0.7000000000000001 ............................
[CV] ............. alpha=1, l1_ratio=0.7000000000000001, total=   0.0s
[CV] alpha=1, l1_ratio=0.7000000000000001 ............................
[CV] ............. alpha=1, l1_ratio=0.7000000000000001, total=   0.0s
[CV] alpha=1, l1_ratio=0.7000000000000001 ............................
[CV] .

[CV] ........................... alpha=10, l1_ratio=0.2, total=   0.0s
[CV] alpha=10, l1_ratio=0.2 ..........................................
[CV] ........................... alpha=10, l1_ratio=0.2, total=   0.0s
[CV] alpha=10, l1_ratio=0.2 ..........................................
[CV] ........................... alpha=10, l1_ratio=0.2, total=   0.0s
[CV] alpha=10, l1_ratio=0.2 ..........................................
[CV] ........................... alpha=10, l1_ratio=0.2, total=   0.0s
[CV] alpha=10, l1_ratio=0.30000000000000004 ..........................
[CV] ........... alpha=10, l1_ratio=0.30000000000000004, total=   0.0s
[CV] alpha=10, l1_ratio=0.30000000000000004 ..........................
[CV] ........... alpha=10, l1_ratio=0.30000000000000004, total=   0.0s
[CV] alpha=10, l1_ratio=0.30000000000000004 ..........................
[CV] ........... alpha=10, l1_ratio=0.30000000000000004, total=   0.0s
[CV] alpha=10, l1_ratio=0.30000000000000004 ..........................
[CV] .

[CV] ........................... alpha=10, l1_ratio=0.9, total=   0.0s
[CV] alpha=10, l1_ratio=0.9 ..........................................
[CV] ........................... alpha=10, l1_ratio=0.9, total=   0.0s
[CV] alpha=10, l1_ratio=0.9 ..........................................
[CV] ........................... alpha=10, l1_ratio=0.9, total=   0.0s
[CV] alpha=10, l1_ratio=0.9 ..........................................
[CV] ........................... alpha=10, l1_ratio=0.9, total=   0.0s
[CV] alpha=10, l1_ratio=0.9 ..........................................
[CV] ........................... alpha=10, l1_ratio=0.9, total=   0.0s
[CV] alpha=10, l1_ratio=0.9 ..........................................
[CV] ........................... alpha=10, l1_ratio=0.9, total=   0.0s
[CV] alpha=10, l1_ratio=0.9 ..........................................
[CV] ........................... alpha=10, l1_ratio=0.9, total=   0.0s
[CV] alpha=10, l1_ratio=0.9 ..........................................
[CV] .

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   34.1s finished


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
                                  l1_ratio=0.5, max_iter=10000000.0,
                                  normalize=False, positive=False,
                                  precompute=False, random_state=None,
                                  selection='cyclic', tol=0.0001,
                                  warm_start=True),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [1e-05, 0.0001, 0.001, 0.01, 0.1, 0.3, 0.5,
                                   0.7, 1, 10],
                         'l1_ratio': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring=None, verbose=2)

In [58]:
elastic_param_search3.best_params_

{'alpha': 0.0001, 'l1_ratio': 1.0}

In [59]:
elastic_param_search3.cv_results_['mean_test_score'].max()

0.849226439985751

## Model submission


In [65]:
y_predict = np.expm1(elastic_param_search3.predict(xtest.loc[:,xtest.columns != 'Id']))

submission = pd.DataFrame({'Id': xtest['Id'], 'SalePrice': y_predict})

submission.to_csv('submission_elasticnet3.csv',index=False)

submission

,Id,SalePrice
0,1461,107522.865372
1,1462,149219.246250
2,1463,191732.438414
3,1464,207324.368786
4,1465,201662.219027
...,...,...
1454,2915,87222.201494
1455,2916,87067.164144
1456,2917,169599.215393
1457,2918,126135.474689
